## 원티드 - 데이터 엔지니어
* 데이터 엔지니어(job-card) -> 세부 페이지 들어가서 JD 스크래핑

### 데이터 엔지니어(job-card) 데이터 가져오기

In [6]:
# 원티드 - 데이터 엔지니어 > 응답률순 정렬
url = 'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=company.response_rate_order&years=-1&locations=all'

In [7]:
import pandas as pd

In [9]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

driver.implicitly_wait(5)
driver.get(url)

last_height = driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height

###### 원티트 페이지
###### 소스를 읽어 BeautifulSoup를 이용해 파싱    
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
container = soup.select('#__next > div.JobList_cn__t_THp > div > div > div.List_List_container__JnQMS > ul')
#print(container[0].prettify())

job_card = container[0].find_all('div', {'class': 'Card_className__u5rsb'})

In [10]:
driver.quit()

In [11]:
# 채용정보 잘 나오는지 확인

print(job_card[0].find('a')['href'])
print(job_card[0].find('a')['data-company-name'])
print(job_card[0].find('a')['data-position-name'])

/wd/83114
티빙(TVING)
DBA


In [12]:
# 몇 개의 채용공고가 있나 확인
print(len(job_card))

456


In [14]:
## 데이터 엔지니어 관련 공고 가져오기( 회사명 - 직무명 -페이지주소 )
job_lists = []
for i in range(len(job_card)):
    address = job_card[i].find('a')['href']
    company_name = job_card[i].find('a')['data-company-name']
    position = job_card[i].find('a')['data-position-name']
    
    ## 각 공고의 페이지 주소는 '/wd/1234' 이런식이니까 'www.wanted.co.kr'을 앞에 넣어주면
    url2 = 'https://www.wanted.co.kr' + address
    
    job_lists.append([company_name, position, url2])

In [15]:
## 위의 결과를 데이터프레임으로 만듦
df1 = pd.DataFrame(job_lists, columns = ['company', 'job','url'])

# csv파일로 저장
df1.to_csv('company.csv')

## 공고 세부 페이지 들어가서 JD 가져오기

In [16]:
job_urls = df1['url']
job_urls

0       https://www.wanted.co.kr/wd/83114
1      https://www.wanted.co.kr/wd/130687
2       https://www.wanted.co.kr/wd/94411
3      https://www.wanted.co.kr/wd/100716
4      https://www.wanted.co.kr/wd/121128
                      ...                
451    https://www.wanted.co.kr/wd/129036
452    https://www.wanted.co.kr/wd/129540
453    https://www.wanted.co.kr/wd/128333
454    https://www.wanted.co.kr/wd/112350
455     https://www.wanted.co.kr/wd/85840
Name: url, Length: 456, dtype: object

In [18]:
###### 채용 공고 주소를 하나씩 가져와서 [회사명, 주요업무, 자격요건, 우대사항] 정리
results = []

for i in range(len(job_urls)):

    url2 = job_urls[i] 
    
    ## 셀레니엄
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

    driver.implicitly_wait(0.1)
    driver.get(url2)
    
    #### 채용공고에 해당되는 박스의 클래스 가져오고 
    jd_box = driver.find_element(By.CLASS_NAME,'JobContent_descriptionWrapper__SM4UD' )

    ## 작성내용은 태그 p로 되어 있으니까 
    contents = jd_box.find_elements(By.TAG_NAME, 'p')
        
    ## 해당 내용들을 정리하면
    company_pr = contents[0]    # 회사명
    to_do = contents[1]    #주요업무
    need_to = contents[2]   #자격요건
    prefer = contents[3]   #우대사항 
    
    ## 회사소개
    value = company_pr.find_element(By.TAG_NAME, 'span')
    content_company_pr = value.get_attribute('innerHTML')

    ## 주요업무
    value = to_do.find_element(By.TAG_NAME, 'span')
    content_to_do = value.get_attribute('innerHTML')

    ## 자격요건
    value = need_to.find_element(By.TAG_NAME, 'span')
    content_need_to = value.get_attribute('innerHTML')

    ## 우대사항
    value = prefer.find_element(By.TAG_NAME, 'span')
    content_prefer = value.get_attribute('innerHTML')

    ## 전부다 불러와서 행으로 들어갈 데이터 만듦
    results.append([content_company_pr,content_to_do,content_need_to, content_prefer, url2])
    
    driver.quit()

## 데이터 프레임 만들기    
df2 = pd.DataFrame(results, columns =['company', 'to_do', 'need_to', 'preference', 'url'])
df2

,company,to_do,need_to,preference,url
0,OTT 서비스의 혁신을 리드하는 크리에이티브!<br><br>No. 1 K-Conte...,- RDBMS/NoSQL 데이터베이스 운영 및 문제해결<br>- 데이터베이스 클러스...,"- Open Source DB 구축/운영 경험, DBA 관련 경력 2년 이상을 보유...",- AWS Cloud 경험을 보유하신 분<br>- NoSQL 구축/운영 및 NoSQ...,https://www.wanted.co.kr/wd/83114
1,"IT 비즈니스 혁신을 선도하는 AI 스타트업, 아인시스에이아이랩(이하 에이아이랩) ...",• 데이터 분석을 위한 알고리즘 및 개발 방향과 범위를 산정해요<br>• 데이터 수...,• 이공계 졸업(예정자) 혹은 관련 경력을 보유하신 분<br>• 네트워크 애플리케이...,• 스타트업 업무경험이 있으신 분<br>• Rust 개발 경험이 있으신 분<br>•...,https://www.wanted.co.kr/wd/130687
2,[회사 소개]<br>이제이엔은 트위치 후원 오버레이 플랫폼 '트윕(TWIP)'과 트...,"담당업무<br>• 데이터 관련 모든 기술적 요소 담당<br>• Kafka, kube...",• Data Engineer 경력 3년 이상 또는 그에 준하는 경력을 보유하신 분<...,• 커뮤니케이션 능력이 우수하신 분<br>• 영어 논문을 보고 기술 재현이 가능하신...,https://www.wanted.co.kr/wd/94411


In [31]:
df2.to_excel('wanted_DE.xlsx', index=False)